<a href="https://colab.research.google.com/github/SankalpC10/Colab_Notebooks/blob/main/LTV_Churn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow.compat.v1 as tf
import tensorflow_probability as tfp
tfd = tfp.distributions

import os

import numpy as np
import pandas as pd
from scipy import stats
import seaborn as sns
from sklearn import model_selection
from sklearn import preprocessing
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
import tensorflow_probability as tfp
import tqdm
from typing import Sequence
import dask.dataframe as dd

In [3]:
# !pip install dask[dataframe]

In [4]:
import os

import numpy as np
import pandas as pd
import tqdm
from sklearn import metrics
from sklearn import model_selection
from sklearn import preprocessing
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
import tensorflow_probability as tfp
from typing import Sequence
import pickle

# install and import ltv
# !pip install -q git+https://github.com/google/lifetime_value
!pip install -q git+https://github.com/seyedrezamirkhani/lifetime_value
import lifetime_value as ltv

  Preparing metadata (setup.py) ... done


In [5]:
%%shell
mkdir ~/.kaggle
echo \{\"username\":\"sankalpchavhan\",\"key\":\"c2c7e3b5079f84cb89cb3557fbcc2aa2\"\} > ~/.kaggle/kaggle.json
pip install kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [10]:
# COMPANY = '104900040'  # @param { isTemplate: true, type: 'string'}
LOSS = 'ziln'  # @param { isTemplate: true, type: 'string'} ['bce', 'ziln']
# LOSS = 'bce'  # @param { isTemplate: true, type: 'string'} ['bce', 'ziln']
# MODEL = 'linear'  # @param { isTemplate: true, type: 'string'} ['linear', 'dnn']
MODEL = 'dnn'  # @param { isTemplate: true, type: 'string'} ['linear', 'dnn']
LEARNING_RATE = 0.0002  # @param { isTemplate: true}
EPOCHS = 400  # @param {type: 'integer'}
DATA_FOLDER = './tmp/acquire-valued-shoppers-challenge' # @param { isTemplate: true, type: 'string'}
OUTPUT_CSV_FOLDER = f'{DATA_FOLDER}/result'  # @param { isTemplate: true, type: 'string'}

In [11]:
# Set it DATA_FOLDER as an environment variable
%env DATA_FOLDER=$DATA_FOLDER

env: DATA_FOLDER=./tmp/acquire-valued-shoppers-challenge


In [12]:
# %%bash
# if [ -e $DATA_FOLDER/transactions.csv ]
# then
#   echo "File already exists, no need to download."
# else
#   rm -rf $DATA_FOLDER
#   mkdir -p $DATA_FOLDER
#   cd $DATA_FOLDER
#   kaggle competitions download -c acquire-valued-shoppers-challenge
#   echo "Unzip file. This may take 10 min."
#   unzip acquire-valued-shoppers-challenge.zip transactions.csv.gz
#   gunzip transactions.csv.gz
# fi

In [13]:
# def load_transaction_data():
#     all_data_filename = f'{DATA_FOLDER}/transactions.csv'
#     req_company_data_filename = f'{DATA_FOLDER}/transactions_company_req.csv'
#     desired_columns = ['id', 'date', 'purchasequantity', 'purchaseamount']
#     if os.path.isfile(req_company_data_filename):
#         df = pd.read_csv(req_company_data_filename, usecols=desired_columns)
#     else:
#         data_list = []
#         chunksize = 10**5
#         for chunk in tqdm.tqdm(pd.read_csv(all_data_filename, chunksize=chunksize)):
#             data_list.append(chunk[desired_columns])
#         df = pd.concat(data_list, axis=0)
#         df.to_csv(req_company_data_filename, index=False)
#     return df

In [14]:
def load_transaction_data():
    all_data_filename = f'{DATA_FOLDER}/transactions.csv'
    req_company_data_filename = f'{DATA_FOLDER}/transactions_company_req.csv'
    desired_columns = ['id', 'date', 'purchasequantity', 'purchaseamount']

    if os.path.isfile(req_company_data_filename):
        df = pd.read_csv(req_company_data_filename, usecols=desired_columns)
    else:
        # Load the data with Dask
        ddf = dd.read_csv(all_data_filename, usecols=desired_columns)
        # Save the filtered data to a new CSV file
        ddf.to_csv(req_company_data_filename, index=False, single_file=True)
        # Load the new CSV as a Pandas DataFrame
        df = pd.read_csv(req_company_data_filename, usecols=desired_columns)

    return df

In [15]:
# def preprocess(df):
#     df = df.query('purchaseamount > 0')

#     # Convert date column to datetime
#     df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')

#     # Calculate the start date for each customer (minimum date per ID)
#     df['start_date'] = df.groupby('id')['date'].transform('min')

#     # Compute calibration values (purchaseamount on start_date)
#     calibration_value = (
#         df.query('date == start_date')
#         .groupby('id')['purchaseamount']
#         .sum()
#         .reset_index()
#     )
#     calibration_value.columns = ['id', 'calibration_value']

#     # Compute holdout values (purchaseamount within a one-year window after start_date)
#     one_year_holdout_window_mask = (
#         (df['date'] > df['start_date']) &
#         (df['date'] <= df['start_date'] + np.timedelta64(365, 'D'))
#     )
#     holdout_value = (
#         df[one_year_holdout_window_mask]
#         .groupby('id')['purchaseamount']
#         .sum()
#         .reset_index()
#     )
#     holdout_value.columns = ['id', 'holdout_value']

#     # Merge calibration and holdout data
#     customer_level_data = calibration_value.merge(holdout_value, how='left', on='id')

#     # Fill missing holdout values with 0
#     customer_level_data['holdout_value'] = customer_level_data['holdout_value'].fillna(0.)

#     # Compute log of calibration value for modeling purposes
#     customer_level_data['log_calibration_value'] = (
#         np.log(customer_level_data['calibration_value']).astype('float32')
#     )

#     # Add a label column for modeling (optional, based on use case)
#     customer_level_data['label'] = customer_level_data['holdout_value'].astype('float32')

#     return customer_level_data

In [23]:
def preprocess_in_chunks(file_path, chunksize=10**5):
    # Temp storage for intermediate results
    calibration_values = {}
    holdout_values = {}

    for chunk in tqdm.tqdm(pd.read_csv(file_path, chunksize=chunksize)):
        # Filter rows where purchaseamount > 0
        chunk = chunk.query('purchaseamount > 0')

        # Convert 'date' column to datetime
        chunk['date'] = pd.to_datetime(chunk['date'], format='%Y-%m-%d')

        # Calculate the start date for each customer in the current chunk
        start_dates = chunk.groupby('id')['date'].min()

        # Update the global calibration dictionary
        for customer_id, start_date in start_dates.items():
            if customer_id not in calibration_values:
                calibration_values[customer_id] = {
                    'start_date': start_date,
                    'calibration_value': 0
                }

        # Update calibration and holdout values for each row in the chunk
        for _, row in chunk.iterrows():
            customer_id = row['id']
            if customer_id in calibration_values:
                start_date = calibration_values[customer_id]['start_date']

                # Update calibration_value for transactions on the start_date
                if row['date'] == start_date:
                    calibration_values[customer_id]['calibration_value'] += row['purchaseamount']

                # Update holdout_value for transactions within the one-year window
                if start_date < row['date'] <= start_date + pd.Timedelta(days=365):
                    holdout_values[customer_id] = (
                        holdout_values.get(customer_id, 0) + row['purchaseamount']
                    )

    # Convert calibration and holdout dictionaries to DataFrames
    calibration_df = pd.DataFrame.from_dict(calibration_values, orient='index').reset_index()
    calibration_df.columns = ['id', 'start_date', 'calibration_value']

    holdout_df = pd.DataFrame.from_dict(holdout_values, orient='index').reset_index()
    holdout_df.columns = ['id', 'holdout_value']

    # Merge calibration and holdout data
    customer_level_data = calibration_df.merge(holdout_df, how='left', on='id')

    # Fill missing holdout values with 0
    customer_level_data['holdout_value'] = customer_level_data['holdout_value'].fillna(0.)

    # Compute log of calibration value
    customer_level_data['log_calibration_value'] = (
        np.log(customer_level_data['calibration_value']).astype('float32')
    )

    # Add label column for modeling
    customer_level_data['label'] = customer_level_data['holdout_value'].astype('float32')

    return customer_level_data

In [21]:
# def load_customer_level_csv():
#     # File to save/load preprocessed customer-level data
#     customer_level_data_file = f'{DATA_FOLDER}/customer_level_data.csv'

#     # Check if the file already exists
#     if os.path.isfile(customer_level_data_file):
#         # Load preprocessed customer-level data
#         customer_level_data = pd.read_csv(customer_level_data_file)
#     else:
#         # Load and preprocess transaction data
#         transaction_data = load_transaction_data()  # No company parameter
#         customer_level_data = preprocess(transaction_data)

#         # Save preprocessed data to file for future use
#         customer_level_data.to_csv(customer_level_data_file, index=False)

#     # Cast numerical columns to appropriate data types for efficiency
#     for num_col in ['log_calibration_value', 'calibration_value', 'holdout_value']:
#         customer_level_data[num_col] = customer_level_data[num_col].astype('float32')

#     return customer_level_data

In [24]:
def load_customer_level_csv():
    # File to save/load preprocessed customer-level data
    customer_level_data_file = f'{DATA_FOLDER}/customer_level_data.csv'

    if os.path.isfile(customer_level_data_file):
        # Load preprocessed data
        customer_level_data = pd.read_csv(customer_level_data_file)
    else:
        # Chunk-based preprocessing
        transaction_file = f'{DATA_FOLDER}/transactions_company_req.csv'
        customer_level_data = preprocess_in_chunks(transaction_file)

        # Save preprocessed data to file for future use
        customer_level_data.to_csv(customer_level_data_file, index=False)

    # Cast numerical columns to appropriate data types for efficiency
    for num_col in ['log_calibration_value', 'calibration_value', 'holdout_value']:
        customer_level_data[num_col] = customer_level_data[num_col].astype('float32')

    return customer_level_data


In [ ]:
customer_level_data = load_customer_level_csv()

Streaming output truncated to the last 5000 lines.
See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['date'] = pd.to_datetime(chunk['date'], format='%Y-%m-%d')
762it [1:53:08,  8.85s/it]<ipython-input-23-b4ff950f0707>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['date'] = pd.to_datetime(chunk['date'], format='%Y-%m-%d')
763it [1:53:17,  9.01s/it]<ipython-input-23-b4ff950f0707>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returni

In [ ]:
customer_level_data

In [18]:
file_path = f'{DATA_FOLDER}/transactions_company_req.csv'

# Read only the first 5 rows
head_data = pd.read_csv(file_path, nrows=5)

print(head_data)

      id        date  purchasequantity  purchaseamount
0  86246  2012-03-02                 1            7.59
1  86246  2012-03-02                 1            1.59
2  86246  2012-03-02                 1            5.99
3  86246  2012-03-02                 1            1.99
4  86246  2012-03-02                 2           10.38
